In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
import os
from os.path import join
from threading import Thread
import time
import gc

from scipy import misc

In [2]:
### Settings ###
source = 'processed_predictions'
output = 'submit.csv'

In [3]:
all_img = os.listdir(source)
all_img = [join(source ,f) for f in all_img]

In [4]:
class getTestData:
    
    def run(self, arr_x, arr_names, img_files, per_array=30):
        
        self.arr_x = arr_x
        self.arr_names = arr_names
        self.img_files = img_files
        self.per_array = per_array
        
        self.thread = Thread(target = self.fill_array)
        self.thread.start()
    
    def fill_array(self):

        while(True):
            if(len(self.arr_x) <3):
                threads = []
                img_to_arr = self.img_files[0:self.per_array]
                del self.img_files[0:self.per_array]
                
                x = [None] * self.per_array

                for i, file in enumerate(img_to_arr):
                    thread = Thread(target=self.read_img, args=(file, x, i))
                    thread.start()
                    threads.append(thread)

                for t in threads:
                    t.join()
                
                self.arr_x.append(x)
                names = [f.split('\\')[-1] for f in img_to_arr]
                self.arr_names.append(names)

                # At the end
                if(self.per_array > len(self.img_files)):
                    self.per_array = len(self.img_files)
                if(len(self.img_files) == 0):
                    break

                gc.collect()

            else:
                time.sleep(1)

    def read_img(self, file, x, loc):
        x[loc] = misc.imread(file, mode='L')

In [5]:
def rle_encode(mask_image):
    pixels = mask_image.flatten()
    # We avoid issues with '1' at the start or end (at the corners of 
    # the original image) by setting those pixels to '0' explicitly.
    # We do not expect these to be non-zero for an accurate mask, 
    # so this should not harm the score.
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs

def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

In [6]:
array_x = []
array_names = []
obj = getTestData()
obj.run(array_x, array_names, all_img, 30)
df = pd.DataFrame(columns=['img', 'rle_mask'])

In [7]:
while (len(all_img) != 0 or len(array_x) != 0):
    if(len(array_x) != 0):
        x = np.array(array_x.pop(0))
        f_names = array_names.pop(0)
        l = []
        for i, x_ in enumerate(x):
            x_[x_ <= 127] = 0
            x_[x_ > 127] = 1
            d = {'img': f_names[i], 'rle_mask': rle_to_string(rle_encode(x_))}
            l.append(d)
        df = df.append(l, ignore_index=True)
    else:
        time.sleep(1)
        
df.to_csv(output, index=False)